In [1]:
# Kinase Enrichment Analysis from NetworKIN Output


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.io import export_svgs
from scipy import stats
import seaborn as sns
from colour import Color
from matplotlib import colors
import scipy.special
from bokeh.layouts import gridplot
from statsmodels.sandbox.stats.multicomp import multipletests


output_notebook()

Loading BokehJS ...

In [167]:
# Raw Data In
networkin_out = pd.read_csv("networKIN_out_Ensembl74_IsoformsRemoved.txt", sep='\t')
groupAnnotations = pd.read_csv("20181008_RapMICData_GroupAnnotations.csv")
groupAnnotations2 = pd.read_csv("20181008_RapMICData_GroupAnnotations_NewGroupings.csv")

# Annotate networkin_out with group annotations
networkin_out["EnsemblID"] = networkin_out["#Name"] + networkin_out["Position"]


In [181]:
len(networkin_out["EnsemblID"].unique())

7879

In [104]:
# Subset all MAPK3 Assignments
networkin_out_MAPK3 = networkin_out.loc[networkin_out["Kinase/Phosphatase/Phospho-binding domain"] == "MAPK3"]

# Subset all MIC Main Effects in GroupAnnotations
MIC_MainEffects_PositiveFC_IDS = groupAnnotations.loc[groupAnnotations["HasKinase_MIC_MainEffect_((VEC+REC)-(VC+RC))>0"]==1]

# Subset all MAPK3 Assignments that are in MIC Main Effect
networkin_out_MAPK3_MICMainEffect = networkin_out_MAPK3[networkin_out_MAPK3["EnsemblID"].isin(MIC_MainEffects_PositiveFC_IDS["ENSEMBL74ID"])]

#Subset All CSNK1A1 Assignment
networkin_out_CSNK1A1 = networkin_out.loc[networkin_out["Kinase/Phosphatase/Phospho-binding domain"] == "CK1alpha"]

#Subset all CSNK1A1 Assignment that are in MIC Main Effect
networkin_out_CSNK1A1_MICMainEffect = networkin_out_CSNK1A1[networkin_out_CSNK1A1["EnsemblID"].isin(MIC_MainEffects_PositiveFC_IDS["ENSEMBL74ID"])]

In [6]:
len(networkin_out_CSNK1A1)

7626

In [131]:
# Plot histogram of MAPK3 MIC Main Effect Scores

p = figure(title="Distribution of MAPK3 Scores in MIC Main Effect", tools = ["save","zoom_in"], x_range =(0,3), y_range=(0,1))

scores1 = networkin_out_MAPK3_MICMainEffect["NetworKIN score"][networkin_out_MAPK3_MICMainEffect["NetworKIN score"] < 3]
print(max(scores1))
#scores1 = networkin_out_CSNK1A1_MICMainEffect["NetworKIN score"]
#print(len(scores1))
scores1_filter = scores1[scores1>0.1]
print(len(scores1_filter))
#print(scores1_filter)
hist1, edges1 = np.histogram(scores1_filter, density=True, bins = 300)

cdf1 = np.cumsum(hist1)

p.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       fill_color="#036564", 
       line_color="#033649")
p.line(edges1[1:], cdf1/cdf1[-1])

print("CDF1[-1]: {}".format(cdf1[-1]))

#print((cdf1/cdf1[-1])[250])

p2 = figure(title="Distribution of All MAPK3 Scores", tools = "save", x_range=(0,3), y_range=(0,1))#,output_backend = "svg")
#p2.output_backend = "svg"
scores2 = networkin_out_MAPK3["NetworKIN score"][networkin_out_MAPK3["NetworKIN score"] <3]
#scores2 = networkin_out_CSNK1A1["NetworKIN score"]

hist2, edges2 = np.histogram(scores2[scores2>0.1], density = True, bins = 300)

cdf2 = np.cumsum(hist2)

p2.quad(top = hist2 / max(hist2),
       bottom = 0,
       left = edges2[1:],
       right = edges2[:-1],
       fill_color="#036564", 
       line_color="red",
       alpha=0.5)
p2.line(edges2[1:], cdf2/cdf2[-1])

p2.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       fill_color="#FF0000", 
       line_color="#033649",
       alpha = 0.5)
p2.line(edges1[1:], cdf1/cdf1[-1], color="red")

print("CDF2[-1]: {}".format(cdf2[-1]))
print(max(scores2))
p2.output_backend = "svg"

p3 = figure(title="Overlayed CDFs", tools = "save", x_range=(0,3), y_range=(0,1))

p3.line(edges1[1:], cdf1/cdf1[-1], color = "blue")
p3.line(edges2[1:], cdf2/cdf2[-1], color = "red")

#show(gridplot(p,p2,p3, ncols=2, plot_width=300, plot_height=300))

#show(p)
#show(p2)

#print("Average scores 1:%s"%np.mean(scores1))
#print("Average scores 2:%s"%np.mean(scores2))

#p.output_backend = "svg"

#export_svgs(p, filename="MAPK3_MICMainEffect.svg")
export_svgs(p2, filename="MAPK3_AllSitesVSMICMainEffect.svg")


#Overlayed Histograms
show(p2)

""""
scores1df = scores1.to_frame()
scores1df["Color"] = 1

scores2df = scores2.to_frame()
scores2df["Color"] = 2

scores = [scores1df, scores2df]
scoresAll = pd.concat(scores)

p4 = figure(title="Distribution of MAPK3 Scores", tools = "save", x_range=(0,3), y_range=(0,1))

histAll, edgesAll = np.histogram(scoresAll, color="Color")

p4.quad(top = histAll/max(histAll),
       bottom = 0,
       left = edgesAll[1:],
       right = edgesAll[:-1])

show(histAll)
#print("CSNK1A1 KS pValue: " + str(stats.ks_2samp(scores1, scores2)[1]))

hist4 = Histogram(df, values='hp', color='cyl',
                  title="df, values='hp', color='cyl'", legend='top_right')
"""

2.9996
494
CDF1[-1]: 103.90689941812137
CDF2[-1]: 103.55182768975877
2.9996


'"\nscores1df = scores1.to_frame()\nscores1df["Color"] = 1\n\nscores2df = scores2.to_frame()\nscores2df["Color"] = 2\n\nscores = [scores1df, scores2df]\nscoresAll = pd.concat(scores)\n\np4 = figure(title="Distribution of MAPK3 Scores", tools = "save", x_range=(0,3), y_range=(0,1))\n\nhistAll, edgesAll = np.histogram(scoresAll, color="Color")\n\np4.quad(top = histAll/max(histAll),\n       bottom = 0,\n       left = edgesAll[1:],\n       right = edgesAll[:-1])\n\nshow(histAll)\n#print("CSNK1A1 KS pValue: " + str(stats.ks_2samp(scores1, scores2)[1]))\n\nhist4 = Histogram(df, values=\'hp\', color=\'cyl\',\n                  title="df, values=\'hp\', color=\'cyl\'", legend=\'top_right\')\n'

In [ ]:
# Plot histogram of MAPK3 MIC Main Effect Scores - With ColumnDataSource

p = figure(title="Distribution of MAPK3 Scores in MIC Main Effect", tools = ["save","zoom_in"], x_range =(0,3), y_range=(0,1))

scores1 = networkin_out_MAPK3_MICMainEffect["NetworKIN score"][networkin_out_MAPK3_MICMainEffect["NetworKIN score"] < 3]
print(max(scores1))
#scores1 = networkin_out_CSNK1A1_MICMainEffect["NetworKIN score"]
#print(len(scores1))
scores1_filter = scores1[scores1>0.1]
print(len(scores1_filter))
#print(scores1_filter)
hist1, edges1 = np.histogram(scores1_filter, density=True, bins = 300)




source = ColumnDataSource 

cdf1 = np.cumsum(hist1)

p.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       fill_color="#036564", 
       line_color="#033649")
p.line(edges1[1:], cdf1/cdf1[-1])

print("CDF1[-1]: {}".format(cdf1[-1]))

#print((cdf1/cdf1[-1])[250])

p2 = figure(title="Distribution of All MAPK3 Scores", tools = "save", x_range=(0,3), y_range=(0,1))#,output_backend = "svg")
#p2.output_backend = "svg"
scores2 = networkin_out_MAPK3["NetworKIN score"][networkin_out_MAPK3["NetworKIN score"] <3]
#scores2 = networkin_out_CSNK1A1["NetworKIN score"]

hist2, edges2 = np.histogram(scores2[scores2>0.1], density = True, bins = 300)

cdf2 = np.cumsum(hist2)

p2.quad(top = hist2 / max(hist2),
       bottom = 0,
       left = edges2[1:],
       right = edges2[:-1],
       fill_color="#036564", 
       line_color="red",
       alpha=0.5)
p2.line(edges2[1:], cdf2/cdf2[-1])

p2.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       fill_color="#FF0000", 
       line_color="#033649",
       alpha = 0.5)
p2.line(edges1[1:], cdf1/cdf1[-1], color="red")

print("CDF2[-1]: {}".format(cdf2[-1]))
print(max(scores2))
p2.output_backend = "svg"

p3 = figure(title="Overlayed CDFs", tools = "save", x_range=(0,3), y_range=(0,1))

p3.line(edges1[1:], cdf1/cdf1[-1], color = "blue")
p3.line(edges2[1:], cdf2/cdf2[-1], color = "red")

#show(gridplot(p,p2,p3, ncols=2, plot_width=300, plot_height=300))

#show(p)
#show(p2)

#print("Average scores 1:%s"%np.mean(scores1))
#print("Average scores 2:%s"%np.mean(scores2))

#p.output_backend = "svg"

#export_svgs(p, filename="MAPK3_MICMainEffect.svg")
export_svgs(p2, filename="MAPK3_AllSitesVSMICMainEffect.svg")


#Overlayed Histograms
show(p2)


In [135]:
# Plot histogram of CSNK1A1 MIC Main Effect Scores

p = figure(title="Distribution of CSNK1A1 Scores in MIC Main Effect", tools = ["save","zoom_in"], x_range =(0,3), y_range=(0,1))
p.output_backend= "svg"
scores1 = networkin_out_CSNK1A1_MICMainEffect["NetworKIN score"][networkin_out_CSNK1A1_MICMainEffect["NetworKIN score"] < 3]
print(max(scores1))
#scores1 = networkin_out_CSNK1A1_MICMainEffect["NetworKIN score"]
#print(len(scores1))
scores1_filter = scores1[scores1>0.1]
print(len(scores1_filter))
#print(scores1_filter)
hist1, edges1 = np.histogram(scores1_filter, density=True, bins = 300)

cdf1 = np.cumsum(hist1)

p.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       color="#036564", 
       line_color="#033649",
      alpha=0.5)
p.line(edges1[1:], cdf1/cdf1[-1])

print("CDF1[-1]: {}".format(cdf1[-1]))

#print((cdf1/cdf1[-1])[250])

p2 = figure(title="Distribution of All CSNK1A1 Scores", tools = "save", x_range=(0,3), y_range=(0,1))

scores2 = networkin_out_CSNK1A1["NetworKIN score"][networkin_out_CSNK1A1["NetworKIN score"] <3]
#scores2 = networkin_out_CSNK1A1["NetworKIN score"]

hist2, edges2 = np.histogram(scores2[scores2>0.1], density = True, bins = 300)

cdf2 = np.cumsum(hist2)

p2.quad(top = hist2 / max(hist2),
       bottom = 0,
       left = edges2[1:],
       right = edges2[:-1],
       fill_color="#036564", 
       line_color="#033649",
       alpha=0.5)
p2.line(edges2[1:], cdf2/cdf2[-1], color = "red")

p2.quad(top = hist1/max(hist1),
       bottom=0, 
       left=edges1[1:], 
       right=edges1[:-1],
       color="red", 
       line_color="red",
      alpha=0.5)
p2.line(edges1[1:], cdf1/cdf1[-1])

p2.output_backend = "svg"

#p3 = figure(title="Overlayed CDFs", tools = "save", x_range=(0,3), y_range=(0,1))
#p3.output_backend = "svg"
#p3.line(edges1[1:], cdf1/cdf1[-1], color = "blue")
#p3.line(edges2[1:], cdf2/cdf2[-1], color = "red")

#show(p)
show(p2)
#show(p3)

#export_svgs(p, filename="CSNK1A1_MICMainEffect.svg")
export_svgs(p2, filename = "CSNK1A1_AllSitesVSMICMainEffect.svg")


print("Average scores 1:%s"%np.mean(scores1))
print("Average scores 2:%s"%np.mean(scores2))

print("CSNK1A1 KS pValue: " + str(stats.ks_2samp(scores1, scores2)[1]))


2.1876
481
CDF1[-1]: 143.74011786689658


Average scores 1:0.2902014598540146
Average scores 2:0.3216140688926997
CSNK1A1 KS pValue: 0.01671167501369561


In [111]:
#Now let's do it for all kinases using the function below

def generateSubsets(clusterName):
    kinases = networkin_out["Kinase/Phosphatase/Phospho-binding domain"].unique()
    KS_values = {}
    for kinase in kinases:
        # Subset all kinase assignments
        networkin_out_filteredForKinase = networkin_out.loc[networkin_out["Kinase/Phosphatase/Phospho-binding domain"] == kinase]
        # Subset assignments in the given group
        group_IDS = groupAnnotations.loc[groupAnnotations[clusterName]==1]
        # Subset all kinase assignments that are in group
        networkin_out_FilteredKinase_byGroup = networkin_out_filteredForKinase[networkin_out_filteredForKinase["EnsemblID"].isin(group_IDS["ENSEMBL74ID"])]
        # Scores for that kinase in all assignments
        scores_All = networkin_out_filteredForKinase["NetworKIN score"]
        # Scores for that kinase in MIC Main Effect
        scores_Group = networkin_out_FilteredKinase_byGroup["NetworKIN score"]
        # Perform KS Test if has values
        if len(scores_All)> 0 and len(scores_Group) > 0:
            KS_values[kinase] = [stats.ks_2samp(scores_All, scores_Group)[1], np.mean(scores_Group) - np.mean(scores_All)]
    
    
    KS_return = pd.DataFrame.from_dict(KS_values, orient='index')
    KS_return.columns = ["p","Delta Average Score"]
    
    reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(KS_return["p"], alpha=0.05, method='fdr_bh')
    KS_return["q_BH"] = pvals_corrected
    KS_return["Reject"] = reject
   
    return KS_return

    



In [112]:
KS_MIC_MainEffect_UpReg = generateSubsets("HasKinase_MIC_MainEffect_((VEC+REC)-(VC+RC))>0")

In [113]:
KS_MIC_MainEffect_UpReg.to_csv("KS_MIC_MainEffect_UpReg.csv")

In [114]:
KS_MIC_MainEffect_DownReg = generateSubsets("HasKinase_MIC_MainEffect_((VEC+REC)-(VC+RC))<0")

In [116]:
KS_MIC_MainEffect_DownReg.to_csv("KS_MIC_MainEffect_DownReg.csv")

In [117]:
KS_RAP_MainEffect_UpReg = generateSubsets("HasKinase_ RAP_MainEffect_((RC+REC)-(VC+VEC))>0")

In [118]:
KS_RAP_MainEffect_UpReg.to_csv("KS_RAP_MainEffect_UpReg.csv")

In [119]:
KS_Rap_MainEffect_DownReg = generateSubsets("HasKinase_ RAP_MainEffect_((RC+REC)-(VC+VEC))<0")

In [120]:
KS_Rap_MainEffect_DownReg.to_csv("KS_Rap_MainEffect_DownReg.csv")

In [121]:
KS_Interaction_UpReg = generateSubsets("HasKinase_Interaction_VEC>0")

In [122]:
KS_Interaction_UpReg.to_csv("KS_Interaction_UpReg.csv")

In [123]:
KS_Interaction_DownReg = generateSubsets("HasKinase_Interaction_VEC<0")

In [124]:
KS_Interaction_DownReg.to_csv("KS_Interaction_DownReg.csv")

In [122]:
p1 = figure(title="Distribution of p Values from KS Test - MIC Main Effect", 
            tools = ["save","zoom_in"], x_range =(0,1), y_range=(0,1))

hist, edges = np.histogram(KS_MIC_MainEffect_UpReg["p"], density=True, bins = 50)

p1.quad(top = hist/max(hist),
        bottom = 0,
        left = edges[1:],
        right = edges[:-1],
        fill_color="#036564", 
        line_color="#033649")


p2 = figure(title="Distribution of q Values from KS Test - MIC Main Effect", 
            tools = ["save","zoom_in"], x_range =(0,1), y_range=(0,1))

hist2, edges2 = np.histogram(KS_allKinasesInMIC_Panda["q_BH"], density=True, bins = 50)

p2.quad(top = hist2/max(hist2),
       bottom = 0,
       left = edges2[1:],
       right = edges2[:-1],
       fill_color="#036564", 
       line_color="#033649")


show(gridplot(p1,p2, ncols=2, plot_width=300, plot_height=300))
KS_allKinasesInMIC_Panda.to_csv("KolmogoraovSmirnovTestonKinases_MIC Main Effect.csv")

#KS_allKinasesInMIC_Panda["q_BH"] = stats.p_adjust(KS_allKinasesInMIC_Panda[""])


In [156]:
# Changed Group Annotations - Rerun enrichment analysis

def performEnrichment_newGroups(clusterName):
    kinases = networkin_out["Kinase/Phosphatase/Phospho-binding domain"].unique()
    KS_values = {}
    for kinase in kinases:
        # Subset all kinase assignments
        networkin_out_filteredForKinase = networkin_out.loc[networkin_out["Kinase/Phosphatase/Phospho-binding domain"] == kinase]
        # Subset assignments in the given group
        group_IDS = groupAnnotations2.loc[groupAnnotations2[clusterName]==1]
        # Subset all kinase assignments that are in group
        networkin_out_FilteredKinase_byGroup = networkin_out_filteredForKinase[networkin_out_filteredForKinase["EnsemblID"].isin(group_IDS["ENSEMBL74ID"])]
        # Scores for that kinase in all assignments
        scores_All = networkin_out_filteredForKinase["NetworKIN score"]
        # Scores for that kinase in MIC Main Effect
        scores_Group = networkin_out_FilteredKinase_byGroup["NetworKIN score"]
        # Perform KS Test if has values
        if len(scores_All)> 0 and len(scores_Group) > 0:
            KS_values[kinase] = [stats.ks_2samp(scores_All, scores_Group)[1], np.mean(scores_Group) - np.mean(scores_All)]
    
    
    KS_return = pd.DataFrame.from_dict(KS_values, orient='index')
    KS_return.columns = ["p","Delta Average Score"]
    
    reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(KS_return["p"], alpha=0.05, method='fdr_bh')
    KS_return["q_BH"] = pvals_corrected
    KS_return["Reject"] = reject
   
    return KS_return





In [157]:
KS_MIC_MainEffect_VCVECq_UpReg = performEnrichment_newGroups("HasKinase_MIC_MainEffectP<0.05_VCvsVECQ<0.05_((VEC+REC)-(VC+RC))>0")

In [161]:
KS_MIC_MainEffect_VCVECq_UpReg.to_csv("KS_MIC_MainEffect_VCVECq_UpReg.csv")

In [158]:
KS_MIC_MainEffect_VCVECq_DownReg = performEnrichment_newGroups("HasKinase_MIC_MainEffectP<0.05_VCvsVECQ<0.05_((VEC+REC)-(VC+RC))<0")

In [162]:
KS_MIC_MainEffect_VCVECq_DownReg.to_csv("KS_MIC_MainEffect_VCVECq_DownReg.csv")

In [159]:
KS_Interaction_VCVECq_UpReg = performEnrichment_newGroups("HasKinase_InteractionP<0.05_VCvsVECQ<0.05_((VEC+REC)-(VC+RC))>0")

In [163]:
KS_Interaction_VCVECq_UpReg.to_csv("KS_Interaction_VCVECq_UpReg.csv")

In [160]:
KS_Interaction_VCVECq_DownReg = performEnrichment_newGroups("HasKinase_InteractionP<0.05_VCvsVECQ<0.05_((VEC+REC)-(VC+RC))<0")

In [164]:
KS_Interaction_VCVECq_DownReg.to_csv("KS_Interaction_VCVECq_DownReg.csv")

In [168]:
KS_RAP_RCvsVCMODq_UpReg = performEnrichment_newGroups("HasKinase_MICMainEffectQ<0.05_RCvsVCMODq<0.05_RVAvg>0")

In [170]:
KS_RAP_RCvsVCMODq_UpReg.to_csv("KS_MIC_RCvsVCMODq_UpReg.csv")

In [171]:
KS_RAP_RCvsVCMODq_DownReg = performEnrichment_newGroups("HasKinase_MICMainEffectQ<0.05_RCvsVCMODq<0.05_RVAvg<0")

In [172]:
KS_RAP_RCvsVCMODq_DownReg.to_csv("KS_MIC_RCvsVCMODq_DownReg.csv")

In [174]:
KS_RAP_RCvsVCPERq_UpReg = performEnrichment_newGroups("HasKinase_MICMainEffectQ<0.05_RCvsVCPERMq<0.05_RVAvg>0")

In [178]:
KS_RAP_RCvsVCPERq_UpReg.to_csv("KS_MIC_RCvsVCPERMq_UpReg.csv")

In [176]:
KS_RAP_RCvsVCPERq_DownReg = performEnrichment_newGroups("HasKinase_MICMainEffectQ<0.05_RCvsVCPERMq<0.05_RVAvg<0")

In [179]:
KS_RAP_RCvsVCPERq_DownReg.to_csv("KS_MIC_RCvsVCPERMq_DownReg.csv")